In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("datasets/nhl.csv")
cities=pd.read_html("datasets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_cities=cities[['Metropolitan area','NHL','Population (2016 est.)[8]']]
    nhl_cities=nhl_cities.set_index('NHL')
    nhl_cities=nhl_cities.rename(index={'RangersIslandersDevils[note 3]':'Rangers',
                        'KingsDucks':'Kings'})
    nhl_cities.loc['Islanders']=None
    nhl_cities.loc['Devils']=None
    nhl_cities.loc['Ducks']=None
    nhl_cities.loc['Islanders']['Metropolitan area']='New York City'
    nhl_cities.loc['Islanders']['Population (2016 est.)[8]']=20153634
    nhl_cities.loc['Devils']['Metropolitan area']='New York City'
    nhl_cities.loc['Devils']['Population (2016 est.)[8]']=20153634
    nhl_cities.loc['Ducks']['Metropolitan area']='Los Angeles'
    nhl_cities.loc['Ducks']['Population (2016 est.)[8]']=13310447
    nhl_cities=nhl_cities.reset_index()
    nhl_cities['NHL']=nhl_cities['NHL'].str.replace('\[.*\]',"")
    nhl_cities=nhl_cities.set_index('Metropolitan area')
    nhl_cities.rename(index={'New York City':'New York','Miami–Fort Lauderdale':'Florida',
                         'Minneapolis–Saint Paul':'Minnesota','Washington, D.C.':'Washington'
                         ,'Dallas–Fort Worth':'Dallas',
                         'San Francisco Bay Area':'San Jose',
                         'Tampa Bay Area':'Tampa Bay'},inplace=True)
    nhl_cities=nhl_cities.reset_index()
    nhl_cities=nhl_cities.set_index('NHL')
    nhl_cities.loc['Golden Knights']['Metropolitan area']='Vegas'
    nhl_cities.loc['Avalanche']['Metropolitan area']='Colorado'
    nhl_cities.loc['Coyotes']['Metropolitan area']='Arizona'
    nhl_cities.reset_index(inplace=True)
    nhl_cities['team']=nhl_cities['Metropolitan area']+' '+nhl_cities['NHL']
    nhl_df=nhl_df[nhl_df['year']==2018]
    nhl_df['team']=nhl_df['team'].str.replace('*','')
    nhl_df=nhl_df.set_index('team')
    nhl_cities=nhl_cities.set_index('team')
    mergeddf=pd.merge(nhl_df,nhl_cities,how='inner',left_index=True,right_index=True)
    mergeddf['W']=pd.to_numeric(mergeddf['W'])
    mergeddf['L']=pd.to_numeric(mergeddf['L'])
    def calculate_ratio(row):
        return row.loc['W']/(row.loc['W']+row.loc['L'])
    mergeddf['ratio']=mergeddf.apply(calculate_ratio,axis='columns')
    mergeddf=mergeddf.reset_index()
    mergeddf=mergeddf.set_index('team')
    nb=(mergeddf.loc['New York Islanders']['ratio']+mergeddf.loc['New York Rangers']['ratio'])/2
    mergeddf.at['New York Islanders','ratio']=nb
    mergeddf.at['New York Rangers','ratio']=nb
    nhl_cities['ratio']=mergeddf['ratio']
    nhl_cities.dropna()
    cities['NHL']=cities['NHL'].str.replace('\[.*\]',"")
    cities=cities.set_index('NHL')
    nhl_cities=nhl_cities.set_index('NHL')
    nhl_cities=nhl_cities.dropna()
    
    population_by_region = nhl_cities['Population (2016 est.)[8]']
    win_loss_by_region = nhl_cities['ratio']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)
nhl_correlation()




FileNotFoundError: [Errno 2] File datasets/nhl.csv does not exist: 'datasets/nhl.csv'